### 1.import gateNU

In [ ]:
from pepnet import GateNU
import tensorflow as tf
from run_config import RunConfig as cfg
class GateNU:
    def __init__(self,
                 hidden_unit,
                 gamma=2.,
                 l2_reg=cfg.dnn_l2_reg,
                 scope='gate_nu'):
        self.hidden_unit = hidden_unit
        self.gamma = gamma
        self.l2_reg = l2_reg
        self.scope = scope
    def __call__(self, inputs):
        with tf.variable_scope(self.scope):
            output = tf.layers.dense(inputs, self.hidden_unit, activation="relu",
                                    kernel_regularizer=tf.contrib.layers.l2_regularizer(self.l2_reg),
                                    name="dense_1")
            output = tf.layers.dense(output, self.hidden_unit, activation="sigmoid",
                                    kernel_regularizer=tf.contrib.layers.l2_regularizer(self.l2_reg),
                                    name="dense_2")
            return self.gamma * output
# def gen_gate_nu_list(inputs, persona, scope='ppnet', share_gate_nu=False, num_task=cfg.num_task, hidden_units=cfg.dnn_hidden_units, l2_reg=cfg.dnn_l2_reg):
#     num_layer = len(hidden_units)
#     task_gate_list = []
#     with tf.variable_scope(scope):
#         concat_input = tf.concat([persona, tf.stop_gradient(inputs)], axis=-1)
#         if share_gate_nu:
#             gate_nu = [GateNU([hidden_unit*num_task, hidden_unit*num_task], l2_reg=l2_reg) for hidden_unit in hidden_units]
#             for i in range(num_layer):
#                 gate = gate_nu[i](concat_input)
#                 gate = tf.split(gate, num_task, axis=1)
#                 task_gate_list.append(gate)
#         else:
#             gate_nu = [ [GateNU([hidden_unit, hidden_unit], l2_reg=l2_reg) for hidden_unit in hidden_units] for _ in range(num_task) ]
#             for task_id in range(num_task):
#                 task_gates = [ gate_nu[task_id][i](concat_input) for i in range(num_layer) ]
#                 task_gate_list.append(task_gates)
#     return task_gate_list

### 2. add feature

In [ ]:
# ppnet
self.usr_v_level = self.mark_tag(input_tensors.get("v_level"), "usr_v_level")
self.usr_activation_level = self.mark_tag(input_tensors.get("usr_activation_level"), "usr_activation_level")
self.usr_lifecycle = self.mark_tag(input_tensors.get("usr_lifecycle"), "usr_lifecycle")
self.purchase_power = self.mark_tag(input_tensors.get("purchase_power"), "purchase_power")
self.ur1 = self.mark_tag(input_tensors.get("ur1"), "ur1")
self.ur2 = self.mark_tag(input_tensors.get("ur2"), "ur2")
self.bu_id = self.mark_tag(input_tensors.get("bu_id"), "bu_id")
self.cate_id = self.mark_tag(input_tensors.get("cate_id"), "cate_id")
self.ir1 = self.mark_tag(input_tensors.get("ir1"), "ir1")
self.item_price_level_13d = self.mark_tag(input_tensors.get("price_level_13d"), "item_price_level_13d")
self.fs4_saro = self.mark_tag(input_tensors.get("fs4_saro"), "fs4_saro")
self.ship_ontime_percent = self.mark_tag(input_tensors.get("ship_ontime_percent"), "ship_ontime_percent")
self.page_idx = self.mark_tag(input_tensors.get("page_idx"), "page_idx")

# 基础特征
netarray_concat = tf.concat(netarray, axis=1)

# 个性化特征层
self.persona = self.mark_tag(
    tf.concat([self.usr_v_level, self.usr_activation_level, self.usr_lifecycle, self.purchase_power, self.ur1, self.ur2, # user
                self._traffic_src_input_layer, self.page_idx, # context
                self.bu_id, self.cate_id, self.is_ad, self.ir1, self.item_price_level_13d, self.fs4_saro, self.ship_ontime_percent], # item
                axis=1),
    "gate_nu_persona_layer"
    )

# 残差连接网络
gate_nu = GateNU(num_hidden_units)(tf.concat([self.persona, tf.stop_gradient(netarray_concat)], axis=-1))
self.gate_nu_summary['Hidden_{}_{}_GateNU'.format(layer_id, name)] = gate_nu
net = net*gate_nu



### 3. summary show

In [ ]:
self.gate_nu_summary = OrderedDict()
self.gate_nu_summary['Hidden_{}_{}_GateNU'.format(layer_id, name)] = gate_nu

for name, var in self.gate_nu_summary.items():
    variable_summaries(model, name, var, mode)
